In [ ]:
# Scripts to do:
# - Refresher
# - Mapper

In [1]:
import pandas as pd

df = pd.read_csv('/Users/kayvongrunigen/switchdrive/baudisgroup/varImportTestSNVs.tsv', sep = '\t', header = 0)


In [3]:
df['snv_type'] = df['variant_class']
# Clean up
df = df[["biosample_id", "variant_id", "callset_id", "individual_id",
    "reference_name", "start", "end", "reference_bases",
    "alternate_bases", "variant_classification", "variant_state_id",
    "specific_so", "aliquot_id", "reference_id", "case_id",
    "sample_id", "snv_type"]]

df.to_csv('/Users/kayvongrunigen/switchdrive/baudisgroup/varImportTestSNVs.tsv', sep = "\t", index = False)  # and create .tsv file in the directory

In [ ]:
#### Issues
# Why do I change variant_type to variant_types
# Why is there \n in the case ids


In [124]:
import pandas as pd
from bycon_helpers import *
from pymongo import MongoClient
from tqdm import tqdm

# Connect to MongoDB
client = MongoClient()
db = client.progenetix
bs = db.biosamples

# Read the data frame
df = pd.read_csv("../temp/mapfile.tsv", sep = "\t",
    header = 0, low_memory = False)

print("Preparation for mapping...")
# Create legacy ids / external references
df["case_id"] = "pgx:TCGA." + df["case_id"]
df["sample_id"] = "pgx:TCGA." + df["sample_id"]

# Create placeholder for variant id, gets created while import
df["variant_id"] = [" "] * len(df)

# Naming convention from progenetix
df["variant_state_id"] = ["SO:0001059"] * len(df)
df["variant_types"] = df["variant_type"]
df["reference_name"] = df["chromosome"].str.slice(start=3)

# Adding sequence ontologies - http://www.sequenceontology.org/browser/
df.loc[df["variant_type"] == "SNP", "specific_so"] = "SO:0001483"
df.loc[df["variant_type"] == "TNP", "specific_so"] = "SO:0002007"
df.loc[df["variant_type"] == "ONP", "specific_so"] = "SO:0002007"
df.loc[df["variant_type"] == "DEL", "specific_so"] = "SO:0000159"
df.loc[df["variant_type"] == "INS", "specific_so"] = "SO:0000667"

# Convert 1-based MAF files to 0-based
# Explanation @ https://www.biostars.org/p/84686/
df.loc[df["variant_type"].isin(["SNP", "TNP", "ONP", "DEL"]), "start"] -= 1
df.loc[df["variant_type"] == "INS", "end"] -= 1

# Generate callset_ids per aliquot for import and map sample_id to biosample_id
print("Starting progenetix mapping...")

biosample_mapping = {}

for aliquot in tqdm(set(df["aliquot_id"])):

    sample_id = df.loc[df["aliquot_id"] == aliquot, "sample_id"].iloc[0]

    if sample_id not in biosample_mapping:
        hit = bs.find({"external_references.id": {"$regex": sample_id},
                        "biosample_status.id": "EFO:0009656"})

        for entry in hit:
            biosample_mapping[sample_id] = (entry["id"], entry["individual_id"])

            break

    biosample_id, individual_id = biosample_mapping.get(sample_id, ("", ""))

    cs_id = generate_id("pgxcs")

    df.loc[df["aliquot_id"] == aliquot,
        ["callset_id", "biosample_id", "individual_id"]] = cs_id, biosample_id, individual_id


# Clean up
df = df[["biosample_id", "variant_id", "callset_id", "individual_id",
    "reference_name", "start", "end", "reference_bases",
    "alternate_bases", "variant_classification", "variant_state_id",
    "specific_so", "hgvsc", "hgvsp", "hgvsp_short", "aliquot_id",
    "reference_id", "case_id", "sample_id", "variant_types",
    "all_effects", "transcript_id", "gene", "feature",
    "feature_type", "hgnc_id", "ensp", "refseq"]]

Preparation for mapping...
Starting progenetix mapping...


100%|█████████████████████████████████████| 10540/10540 [51:20<00:00,  3.42it/s]


In [130]:
len(variants_in_db) == len(df.dropna(subset=['biosample_id']))

True

In [125]:
variants_in_db = df.dropna(subset = ["biosample_id"])
new = df[df["biosample_id"].isna()]
print(new)

Empty DataFrame
Columns: [biosample_id, variant_id, callset_id, individual_id, reference_name, start, end, reference_bases, alternate_bases, variant_classification, variant_state_id, specific_so, hgvsc, hgvsp, hgvsp_short, aliquot_id, reference_id, case_id, sample_id, variant_types, all_effects, transcript_id, gene, feature, feature_type, hgnc_id, ensp, refseq]
Index: []

[0 rows x 28 columns]


In [82]:
import pandas as pd
from pymongo import MongoClient
from tqdm import tqdm

# Connect to MongoDB
client = MongoClient()
db = client.progenetix
bs = db.biosamples

# Read the data frame
df = pd.read_csv('../temp/mapfile.tsv', sep='\t',
    header=0, low_memory = False)


all_effects = pd.DataFrame()
columns = ['Symbol','Consequence','HGVSp_Short','Transcript_ID','RefSeq','HGVSc','Impact','Canonical','Sift',
          'PolyPhen','Strand']
for col in columns:
    all_effects[col] = []

effects = []
for effect in df['all_effects']:
    info = effect.split(';')
    for eff in info:
        effects.append(eff.split(','))

for eff in effects:
    all_effects.loc[len(all_effects)] = eff

all_effects

In [2]:
import os
from pymongo import MongoClient
import bycon
import pandas as pd
import time, base36

# Prepare mongodb tools
client = MongoClient()
db = client.progenetix
bs = db.biosamples

In [3]:
df = pd.read_csv('../temp/mappingfile.tsv', sep = '\t')

In [16]:
df.keys()

Index(['biosample_id', 'variant_id', 'callset_id', 'chromosome', 'start',
       'end', 'strand', 'reference_bases', 'alternate_bases', 'hgvsc',
       'variant_classification', 'variant_type', 'hgvsp', 'hgvsp_short',
       'aliquot_id', 'reference_id', 'case_id', 'sample_id'],
      dtype='object')

In [14]:
dd_filtered = df.loc[df['variant_type'] == 'SNP', ['hgvsc', 'reference_bases',
                                                   'alternate_bases', 'start', 'end']].copy()
INS = df.loc[df['variant_type'] == 'INS', ['start', 'end', 'reference_bases', 'alternate_bases']].copy()
# Drop rows with missing values in HGVSC
dd_filtered.dropna(subset=['hgvsc'], inplace=True)

# Extract the last character from HGVSC and assign it to a new column 'HGVSC_ref_base'
dd_filtered['HGVSC_ref_base'] = dd_filtered['hgvsc'].str.split('>').str[0].str[-1]
dd_filtered['HGVSC_alt_base'] = dd_filtered['hgvsc'].str.split('>').str[1]


# Reset the index if necessary
dd_filtered.reset_index(drop=True, inplace=True)

compare = dd_filtered[['hgvsc', 'start', 'end', 'HGVSC_ref_base', 'reference_bases', 'HGVSC_alt_base', 'alternate_bases']]
issues = compare[compare['reference_bases'] != compare['HGVSC_ref_base']]
correct = compare[compare['reference_bases'] == compare['HGVSC_ref_base']]

In [15]:
correct

,hgvsc,start,end,HGVSC_ref_base,reference_bases,HGVSC_alt_base,alternate_bases
1,c.103G>C,226064454,226064454,G,G,C,C
2,c.658A>T,26279162,26279162,A,A,T,T
3,c.1380C>A,31266476,31266476,C,C,A,A
4,c.3484C>A,98303765,98303765,C,C,A,A
5,c.2510T>A,124764127,124764127,T,T,A,A
...,...,...,...,...,...,...,...
2424419,c.315G>T,106612395,106612395,G,G,T,T
2424420,c.813G>A,114906851,114906851,G,G,A,A
2424422,c.165T>A,139537086,139537086,T,T,A,A
2424423,c.56G>T,153536303,153536303,G,G,T,T


In [21]:
for df in df_list:
    diff = existing_data.compare(df, align_axis=0)

diff[0]

ValueError: Can only compare identically-labeled DataFrame objects